#Please run this .ipynb in Google Colab with an associated Google Drive account. You may want to consult the following two links to get more information about how to get FREE access to these resources.
https://www.google.com/drive/

https://colab.research.google.com/

##(i) Here we install the onnxruntime library

In [ ]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##(ii) Here we import all necessary libraries
Please put all your files at a desired location. For example, under "/content/drive/MyDrive/Colab Notbooks/USC_BA_estimator/" folder and load our packages. 



In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import onnxruntime as rt
import numpy as np
import matplotlib.pyplot as plt
import os 
import skimage as ski
import skimage.feature
import matplotlib.pyplot as plt
import nibabel                      as nib
import nibabel.freesurfer.mghformat as mgh
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/USC_BA_estimator/3D-CNN')
from scipy.ndimage import zoom
from glob import glob
import time
import datetime
import pandas as pd
from preprocess import processmgz
import savepath

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##(iii) Preprocessing the data. Convert the mgz format into npy.
"path_of_mgz" is the path to the folder which contains the mgz files. \\
The pre-processed mgz file is saved in this format "brain-year-month-day_hour-min-secs.npy". \\
The pre-processed mgz file will be saved under the path of "path_of_mgz". 

In [ ]:
path_of_mgz = "/content/drive/MyDrive/Colab Notebooks/test_brain_age/" # path to the folder (brain.mgz)
brains = processmgz(path_of_mgz)
time_split = time.ctime().split(" ")
brain_save_path=savepath.filename_brainnpy(path_of_mgz, time_split)
np.save(brain_save_path, brains)

##(iv) Load all necessary files and use our models to make the prediction
Please load our onnx model under "output_path". \\
The predicted BA will be saved as csv files under "path_of_mgz". \\
The csv file is saved in this format "BA-year-month-day_hour-min-sec.csv"

In [ ]:
providers = ['CPUExecutionProvider']
output_path = "/content/drive/MyDrive/Colab Notebooks/s3downloads/github/3dcnn_v_1.0.onnx" #Pls change the path and load our onnx model
output_names = ['dense_1']
m = rt.InferenceSession(output_path, providers=providers)

x = brains
onnx_pred = m.run(output_names, {"input": x})
onnx_pred = onnx_pred[0]+22
#
BA_save_path_npy, BA_save_path_csv = savepath.filename_pred(path_of_mgz, time_split)
np.save(BA_save_path_npy, onnx_pred)
pd.DataFrame(onnx_pred).to_csv(BA_save_path_csv)